In [3]:
# Need to make dataset smaller

# First filter out books with less than 100 reviews
with open("goodreads_interactions.csv") as books_fo:
    books_counts = {}
    i = 0
    line = books_fo.readline()
    line = books_fo.readline()
    while line:
        i += 1
        if i % 10**7 == 0:
            print(i)
        split_line = line.split(",")
        book = split_line[1]
        if book in books_counts:
            books_counts[book] +=1
        else:
            books_counts[book] = 1
        line = books_fo.readline()

1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000
11000000
12000000
13000000
14000000
15000000
16000000
17000000
18000000
19000000
20000000
21000000
22000000
23000000
24000000
25000000
26000000
27000000
28000000
29000000
30000000
31000000
32000000
33000000
34000000
35000000
36000000
37000000
38000000
39000000
40000000
41000000
42000000
43000000
44000000
45000000
46000000
47000000
48000000
49000000
50000000
51000000
52000000
53000000
54000000
55000000
56000000
57000000
58000000
59000000
60000000
61000000
62000000
63000000
64000000
65000000
66000000
67000000
68000000
69000000
70000000
71000000
72000000
73000000
74000000
75000000
76000000
77000000
78000000
79000000
80000000
81000000
82000000
83000000
84000000
85000000
86000000
87000000
88000000
89000000
90000000
91000000
92000000
93000000
94000000
95000000
96000000
97000000
98000000
99000000
100000000
101000000
102000000
103000000
104000000
105000000
106000000
107000000
108000000
109000000
110000000
11100000

In [8]:
final_book_set = set()
for book in books_counts:
    if books_counts[book] >= 100:
        final_book_set.add(book)

In [9]:
len(final_book_set)

275555

In [6]:
len(books_counts)

2360650

In [13]:
# Then filter out users with less than 10 review
with open("goodreads_interactions.csv") as books_fo:
    user_review_count = {}
    i = 0
    line = books_fo.readline()
    line = books_fo.readline()
    while line:
        i += 1
        if i % 10**7 == 0:
            print(i)
        split_line = line.split(",")
        user = split_line[0]
        if user in user_review_count:
            user_review_count[user] +=1
        else:
            user_review_count[user] = 1
        line = books_fo.readline()

10000000
20000000
30000000
40000000
50000000
60000000
70000000
80000000
90000000
100000000
110000000
120000000
130000000
140000000
150000000
160000000
170000000
180000000
190000000
200000000
210000000
220000000


In [14]:
final_user_set = set()
for user in user_review_count:
    if user_review_count[user] >= 10:
        final_user_set.add(user)

In [15]:
len(user_review_count)

876145

In [16]:
len(final_user_set)

772730

In [30]:
## Now we are to create our matrix to do SVD on
user_id_look_up = {}
for i,val in enumerate(final_user_set):
    user_id_look_up[val] = i

In [133]:
book_id_look_up = {}
reverse_book_id_look_up = {}
for i, val in enumerate(final_book_set):
    book_id_look_up[val] = i
    reverse_book_id_look_up[i] = val

In [40]:
import numpy as np
from scipy.sparse import lil_matrix, save_npz

In [36]:
review_matrix = lil_matrix((len(final_user_set), len(final_book_set)), 
                          dtype = np.int8)

In [39]:
# Now we will populate this matrix:
with open("goodreads_interactions.csv") as books_fo:
    i = 0
    review_count = 0
    line = books_fo.readline()
    line = books_fo.readline()
    while line:
        i += 1
        if i % 10**7 == 0:
            print("iterations ",i)
        split_line = line.split(",")
        user = split_line[0]
        book = split_line[1]
        rating = split_line[3]
        if book in final_book_set and user in final_user_set:
            review_count += 1
            if review_count % 10**6 == 0:
                print("reviews ", review_count)
            review_matrix[user_id_look_up[user], book_id_look_up[book]] = int(rating)
        line = books_fo.readline()

reviews  1000000
reviews  2000000
reviews  3000000
reviews  4000000
reviews  5000000
reviews  6000000
reviews  7000000
reviews  8000000
iterations  10000000
reviews  9000000
reviews  10000000
reviews  11000000
reviews  12000000
reviews  13000000
reviews  14000000
reviews  15000000
reviews  16000000
reviews  17000000
iterations  20000000
reviews  18000000
reviews  19000000
reviews  20000000
reviews  21000000
reviews  22000000
reviews  23000000
reviews  24000000
reviews  25000000
iterations  30000000
reviews  26000000
reviews  27000000
reviews  28000000
reviews  29000000
reviews  30000000
reviews  31000000
reviews  32000000
reviews  33000000
reviews  34000000
iterations  40000000
reviews  35000000
reviews  36000000
reviews  37000000
reviews  38000000
reviews  39000000
reviews  40000000
reviews  41000000
reviews  42000000
iterations  50000000
reviews  43000000
reviews  44000000
reviews  45000000
reviews  46000000
reviews  47000000
reviews  48000000
reviews  49000000
reviews  50000000
revi

In [42]:
review_matrix_csr = review_matrix.tocsr()

In [43]:
save_npz('./sparse_review_matrix.npz', review_matrix_csr)

In [107]:
from sklearn.neighbors import NearestNeighbors

In [44]:
from scipy.sparse.linalg import svds

In [128]:
good_reads_id_lookup = {}
reverse_good_id_lookup = {}
with open("book_id_map.csv") as id_grid_fo:
    line = id_grid_fo.readline()
    line = id_grid_fo.readline()
    while line:
        split_line = line.split(",")
        good_reads_id_lookup[split_line[1].strip()] = split_line[0]
        reverse_good_id_lookup[split_line[0]] = split_line[1].strip()
        line = id_grid_fo.readline()

In [149]:
reverse_book_id_look_up

{0: '119761',
 1: '164434',
 2: '90718',
 3: '11104',
 4: '15909',
 5: '136024',
 6: '379435',
 7: '328976',
 8: '350971',
 9: '5867',
 10: '194570',
 11: '181843',
 12: '271653',
 13: '92907',
 14: '233618',
 15: '245157',
 16: '19448',
 17: '269583',
 18: '89127',
 19: '342420',
 20: '725306',
 21: '128439',
 22: '251712',
 23: '587089',
 24: '76450',
 25: '382325',
 26: '173426',
 27: '373930',
 28: '196596',
 29: '441156',
 30: '748602',
 31: '10456',
 32: '167452',
 33: '7658',
 34: '489239',
 35: '788',
 36: '91342',
 37: '521834',
 38: '95779',
 39: '63318',
 40: '33574',
 41: '231467',
 42: '257350',
 43: '213490',
 44: '540609',
 45: '145972',
 46: '740706',
 47: '4230',
 48: '107218',
 49: '226178',
 50: '664835',
 51: '170309',
 52: '247007',
 53: '434182',
 54: '132384',
 55: '139425',
 56: '396435',
 57: '727821',
 58: '338651',
 59: '595347',
 60: '406746',
 61: '118032',
 62: '248524',
 63: '74832',
 64: '192033',
 65: '29222',
 66: '76321',
 67: '118984',
 68: '625201',

In [131]:
reverse_good_id_lookup

{'0': '34684622',
 '1': '34536488',
 '2': '34017076',
 '3': '71730',
 '4': '30422361',
 '5': '33503613',
 '6': '33517540',
 '7': '34467031',
 '8': '6383669',
 '9': '486625',
 '10': '19161852',
 '11': '32620335',
 '12': '13152847',
 '13': '25735618',
 '14': '24375664',
 '15': '35097384',
 '16': '34974754',
 '17': '836610',
 '18': '6565837',
 '19': '25899336',
 '20': '26114127',
 '21': '18245960',
 '22': '6392944',
 '23': '22078596',
 '24': '6644782',
 '25': '46164',
 '26': '27833614',
 '27': '10419045',
 '28': '255127',
 '29': '28251002',
 '30': '23168277',
 '31': '32075825',
 '32': '30622471',
 '33': '22591134',
 '34': '31580409',
 '35': '133394',
 '36': '23848190',
 '37': '16130',
 '38': '30139664',
 '39': '32191710',
 '40': '29939161',
 '41': '45252',
 '42': '32283423',
 '43': '43615',
 '44': '29905580',
 '45': '18364442',
 '46': '873993',
 '47': '26721984',
 '48': '8879516',
 '49': '5526',
 '50': '2279538',
 '51': '5043',
 '52': '13543159',
 '53': '18667753',
 '54': '23149647',
 '55

In [138]:
review_matrix_csr_float = review_matrix_csr.astype(float)
u, s, vT = svds(review_matrix_csr_float, k=18)

In [139]:
u.shape # this is the user feature space

(772730, 18)

In [140]:
vT.shape # this is the book feature space 

(18, 275555)

In [202]:
v = vT.transpose()
np.save("v_matrix.npy", v)

In [147]:
nbrs = NearestNeighbors(n_neighbors=10, algorithm='ball_tree').fit(v)

In [154]:
import json
book_name_lookup = {}
with open("goodreads_books.json") as fo:
    line = fo.readline()
    while line:
        json_line = json.loads(line)
        book_name_lookup[json_line["book_id"]] = json_line["title"]
        line = fo.readline()

In [177]:
# So we will now create a centroid of my fave books and see what is the closest 10 books to that centroid
# My Book list:
my_book_list = {
    "The Outer Limits of Reason":"17841838",
    "Complexity":"337123",
    "Reinventing the sacred":"2029664",
    "The Emperors New Mind":"179744",
    "Shadows of the Mind":"187100",
    "Antrifragile":"13530973",
    "The Book of Why":"36204378",
    "The Enigma of Reason":"32336635",
    "Thinking Fast and Slow":"11468377",
    "A random walk down wall street":"40242274",
    "Good Natured":"401682"
}
my_book_list = {
    "The Outer Limits of Reason":"17841838",
    "Complexity":"337123",
    "Complexity 2":"225361",
    "Reinventing the sacred":"2029664",
}
vectors = []
for book in my_book_list:
    sparse_index = book_id_look_up.get(good_reads_id_lookup.get(my_book_list[book]))
    if sparse_index is not None:
        vectors.append(vT[:, sparse_index])
    

In [178]:
centroid = np.dstack(vectors).squeeze().mean(axis=1).reshape(-1,1).transpose()

In [179]:
closest_neighbors = nbrs.kneighbors(centroid,return_distance=False)

In [180]:
for n in closest_neighbors.flatten():
    print(n, book_name_lookup[reverse_good_id_lookup[reverse_book_id_look_up[n]]])

123515 Nothing: A Very Short Introduction
135571 Higgs Discovery: The Power of Empty Space
147877 The Design of Design: Essays from a Computer Scientist
96836 Coup d'État: A Practical Handbook
89451 Letters from a Self-Made Merchant to His Son
146873 Neurophilosophy: Toward a Unified Science of the Mind/Brain
137830 The Pursuit of Power: Europe 1815-1914
161249 Life's Ratchet: How Molecular Machines Extract Order from Chaos
208274 Astonishing Hypothesis: The Scientific Search for the Soul
65965 Stalin's General: The Life of Georgy Zhukov


In [181]:
# Two things we need to save:
# The v Matrix
# The lookup for names to their index in the v matrix and reverse lookup

In [184]:
index_to_name = {}
name_to_index = {}
for i in reverse_book_id_look_up:
    index_to_name[i] = book_name_lookup[reverse_good_id_lookup[reverse_book_id_look_up[i]]]
    name_to_index[book_name_lookup[reverse_good_id_lookup[reverse_book_id_look_up[i]]]] = i


In [189]:
with open("book_to_index.json", "w") as fo:
    json.dump(name_to_index,fo)

In [194]:

sorted_books = sorted(books_counts.items(), key=lambda x:x[1], reverse=True)

In [196]:
top_12k_book_names = {}
for i,num in sorted_books[:12000]:
    top_12k_book_names[book_name_lookup[reverse_good_id_lookup[i]]] = i

In [198]:
with open("top_12kbook_to_index.json", "w") as fo:
    json.dump(top_12k_book_names,fo)

In [190]:
with open("index_to_book.json", "w") as fo:
    json.dump(index_to_name,fo)